In [7]:
import numpy as np
from matplotlib import pyplot
import cv2
from numpy import linalg as LA
import os
from plyfile import PlyData, PlyElement
from sklearn.decomposition import PCA

In [8]:
class Eigen3DFace():
    def __init__(self,n_subjects,red_dim):
        self.n_subjects = n_subjects
        self.red_dim=red_dim
        
    def ply_write(self,inference, save_dir, filename):
        vertex = len(inference)
        data_path = 'face.txt'
        faces = [face.rstrip() for face in open(data_path)]

        statements = [
            "ply",
            'format ascii 1.0',
            'element vertex ' + str(vertex),
            'property double x',
            'property double y',
            'property double z',
            'property uchar red',
            'property uchar green',
            'property uchar blue',
            'element face '+str(len(faces)),
            'property list uchar int vertex_indices',
            'end_header'
        ]

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        statements = statements
        f = open(save_dir + filename, 'w')
        for line in statements:
            f.write("%s\n" % line)
        for line in inference:
            f.write("%s\n" % (" ".join(map(str, (line))) + ' ' + "0 0 0"))
        for line in faces:
            f.write("%s\n" % line)

        f.close()
        return 0
    
    def pca(self,flatten_data):
        ##Apply PCA
        pca = PCA(n_components=self.red_dim)
        pca.fit(flatten_data)
        return pca.components_.T

    
    def read_ply(self,file_path):
            plydata = PlyData.read(file_path).elements[0].data
            return np.array(plydata.tolist())[:, 0:3]
        
    def data_loader(self,data_path):
        print('Loading the data. This can take some minutes ...')
        files = os.listdir(data_path)
        files.sort()
        data = [self.read_ply(data_path+d) for d in files[:n_subjects]]
        return np.array(data)
    
    def fitting(self,data_reduced,target,mean,n_points):
        
        ##Compute pseudo-inverse of data_reduced
        piout = np.linalg.pinv(data_reduced)
        
        ##Solve equation (AX + mean = target for X)
        x = np.dot(piout, (target-mean.reshape(-1,1)))
        variances_of_faces = np.dot(data_reduced,x).reshape(n_points, 3)
        return variances_of_faces
    
    #def save_eigen3Dfaces(self,data_reduced,mean):
    #    for i in range(len(data_reduced.T)):
    #        self.ply_write(data_reduced.T[i].reshape(n_points,3)+mean.reshape(n_points, 3),'./','eigen'+str(i)+'.ply')
    #    return 0
    

In [13]:
if __name__ == '__main__':

    data_path = './plys/'
    save_path = './'
    n_subjects=130
    red_dim = 5 #dimentionality after PCA. It has to me smaller than 'n_subjects'!!!
    e3f = Eigen3DFace(n_subjects,red_dim)
    fitting_target = '0140.ply'
    
    ##Load the ply data
    data = e3f.data_loader(data_path)
    n_points = data.shape[1]
    
    ##Create column vectors from data
    flatten_data = data.reshape(n_subjects,-1)
    
    ##Compute mean 3D face from the data
    mean = np.average(flatten_data,0)
    
    ##Apply PCA
    data_reduced = e3f.pca(flatten_data)
    
    ##Load the target 3D face data
    target = e3f.read_ply(data_path + fitting_target).reshape(1,-1).T
    
    ###Fitting stage###
    variances_of_faces = e3f.fitting(data_reduced,target,mean,n_points)
    
    ##save the result and mean face
    e3f.ply_write(variances_of_faces+mean.reshape(n_points, 3),save_path,'result.ply')
    e3f.ply_write(mean.reshape(n_points, 3),save_path,'mean.ply')

Loading the data. This can take some minutes ...
